In [5]:
import sys
sys.path.append("/root/projects/kinsovet")

In [6]:
import os
import json
import time
import logging
import heapq
from datetime import datetime
from pathlib import Path

import requests
from groq import Groq
from dotenv import load_dotenv

load_dotenv("/root/projects/kinsovet/data_collector/.env")

from data_collector.collector import get_movie_from_api, PocketBaseClient

In [7]:
# Конфигурация
POISKKINO_API_URL = "https://api.poiskkino.dev"
POISKKINO_API_KEY = os.environ["POISKKINO_API_KEY"]
POCKETBASE_URL = os.environ["POCKETBASE_URL"]
POCKETBASE_ADMIN_EMAIL = os.environ["POCKETBASE_ADMIN_EMAIL"]
POCKETBASE_ADMIN_PASSWORD = os.environ["POCKETBASE_ADMIN_PASSWORD"]
GROQ_API_KEY = os.environ["GROQ_API_KEY"]
GROQ_MODEL = "openai/gpt-oss-120b"

In [ ]:
pb = PocketBaseClient(POCKETBASE_URL)
pb.authenticate(POCKETBASE_ADMIN_EMAIL, POCKETBASE_ADMIN_PASSWORD)

In [ ]:
existing_records = pb.get_all_records("movie_lists")

In [ ]:
len(existing_records)

In [15]:

movie_data = get_movie_from_api(258687)

2026-02-13 14:01:48,254 - INFO - Запрос фильма 258687: https://api.poiskkino.dev/v1.4/movie/258687
2026-02-13 14:01:49,229 - ERROR - Ошибка API: 503 - no healthy upstream


In [14]:
movie_data['type']

'movie'

In [ ]:
description = movie_data['description']

In [ ]:
client = Groq(api_key=GROQ_API_KEY)

In [ ]:
completion = client.chat.completions.create(
    model=GROQ_MODEL,
    messages=[
        {
            "role": "system",
            "content": (
                "Перепиши описание фильма, чтобы я мог использовать его в коммерческих целях, "
                "не нарушая авторские права на исходное описание. Не надо добавлять никаких приписок: "
                "я должен иметь возможность просто скопировать твой ответ и вставить на сайт. "
                "Сделай описание простым, коротким, понятным, сухим, без сложных слов и конструкций."
            ),
        },
        {
            "role": "user",
            "content": description,
        },
    ],
    temperature=1,
    max_completion_tokens=8192,
    top_p=1,
    reasoning_effort="medium",
    stream=False,
    stop=None,
)

In [ ]:
completion.choices[0].message.content

In [ ]:
result = ""
for chunk in completion:
    result += chunk.choices[0].delta.content or ""

In [ ]:
result

In [ ]:
completion